<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [81]:
%%time

from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess


import matplotlib.pyplot as plt

import pandas as pd

import pyLDAvis.gensim

import re

import seaborn as sns

from sklearn.feature_extraction import text 
import spacy
from spacy.tokenizer import Tokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline

import squarify


import warnings


nlp = spacy.load("en_core_web_lg")
warnings.filterwarnings("ignore",category=DeprecationWarning)
nlp.Defaults.stop_words.add(" ")

Wall time: 9.18 s


In [39]:
# Import Dataset, drop non-English entries
yelp = pd.read_json('./data/review_sample.json', lines=True)
yelp = yelp.drop([9889, 6204,  469, 6311], axis=0)

In [40]:
print(yelp.shape)
yelp.head()

(9996, 9)


,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [41]:
# Create tokenizer function, using nlp.tokenizer for sped up time

def tokenize(doc):
    doc = nlp.tokenizer(doc)
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]



In [42]:
# Create new column with tokens
yelp['tokens'] = yelp['text'].apply(tokenize)
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[BEWARE, FAKE, FAKE, FAKE, small, business, Lo..."
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA,"[Came, lunch, Togo, Service, quick, Staff, fri..."
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA,"[Vegas, dozen, time, step, foot, Circus, Circu..."
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[go, night, close, street, party, well, actual..."
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA,"[3.5, 4, star, , bad, price, $, 12.99, lunch, ..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [43]:
# Custom Stop Words
added = [' ', ""]
stop_words = text.ENGLISH_STOP_WORDS.union(added)

# Fit+transforming tfidf vectorizer
tfidf = TfidfVectorizer(stop_words=stop_words, max_df=0.99, min_df=0.01, ngram_range=(1,2))
tfidf_matrix = tfidf.fit_transform(yelp['text'])

In [44]:
tfidf_df = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf.get_feature_names())
print(tfidf_df.shape)
tfidf_df.head()
# Only 847 columns... Not much to work with

(9996, 847)


,00,10,10 minutes,100,11,12,15,15 minutes,20,20 minutes,...,wow,wrong,year,years,years ago,yelp,yes,young,yum,yummy
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.166001,0.211225,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.122784,0.0,0.0,0.0,0.0,0.154766,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.206155,0.184696,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [45]:
# fitting NN model on our tfidf
nn = NearestNeighbors(n_neighbors=5, algorithm="kd_tree")
nn.fit(tfidf_df)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [46]:
fake_review = ['''
This product is the WORST! I bought it to replace the one I bought at Harbor Freight, and I tried to use it and it broke after the second use! It's complete garbage, DO NOT BUY!
''']

In [47]:
new_matrix = tfidf.transform(fake_review)
neighbors = nn.kneighbors(new_matrix.todense(), n_neighbors=10)

In [48]:

for ii in neighbors[1][0]:
    print(yelp['text'].iloc[ii][:500])
    print("-----------")

My second home. Of course you have to be careful what you buy because the stuff is made very cheaply but for home use most everything is great. I have bent their hand wrenches with my hands. The ends opened up but I have yet to break or even scar one of their impact sockets. My expensive airless paint sprayer died so I figured I would try theirs. At a $160 bucks not only was it just as good or better than my high dollar one cleaning the Harbor freight one was much easier. Granted if I tried to u
-----------
Huge collection of clothes. Especially for the fur coat! I bought nothing but suggest you to come and take a look if you are a vintage lover.
-----------
Talia was so helpful! I love my new bathing suit and I even bought the cutest romper! In love.
-----------
One day last week, I arrived at school (ASU) early, but needed to use the restroom, so I drove to the closest Starbucks (since I was still in my car) to find that it was only a drive thru. Luckily I saw Essence Bakery Café rig

In [49]:
# Certainly an interesting range of reviews... Some I see being similar, but others are completely the opposite...
# Will test later, perhaps adding specific product names or places

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [50]:
target = yelp['stars']
X_train = yelp['text']
print(len(X_train), len(target))

9996 9996


In [51]:


vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
# clf = LogisticRegression(random_state=42)   # Score in 50s
clf = SGDClassifier(random_state=42)   # Score 0.62

pipe = Pipeline([('vect', vect), ('clf', clf)])


In [52]:
parameters = {
    'vect__max_df': (0.75, 0.90, 0.95, 1.0),
    #'vect__solver': ('newton-cg', 'lbfgs', 'sag', 'saga')
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, target)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   19.7s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [53]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'vect__max_df': 0.75}
0.621548524262131


In [82]:
cvect = CountVectorizer(stop_words='english', ngram_range=(1,2))
# clf = LogisticRegression(random_state=42)   # Score in 50s
clf2 = SGDClassifier(random_state=42)   # Score 0.62

pipe2 = Pipeline([('vect', cvect), ('clf', clf2)])
grid_search2 = GridSearchCV(pipe2, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search2.fit(X_train, target)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   15.8s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        prep

In [84]:
#Cvect score is worse
print(f'Score With Count Vectorizer: {grid_search2.best_score_}')

Score With Count Vectorizer: 0.5990389694847424


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [66]:
df = pd.DataFrame()
df['tokens'] = yelp['text'].apply(tokenize)


Learn the vocubalary of the yelp data:

In [67]:
id2word = corpora.Dictionary(df['tokens'])

Create a bag of words representation of the entire corpus

In [68]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

Your LDA model should be ready for estimation: 

In [73]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 7 # You can change this parameter
                  )

In [74]:
lda.print_topics()

[(0,
  '0.051*"" + 0.012*"place" + 0.011*"food" + 0.011*"good" + 0.010*"come" + 0.007*"time" + 0.007*"order" + 0.007*"go" + 0.007*"service" + 0.006*"great"'),
 (1,
  '0.053*"" + 0.011*"place" + 0.009*"time" + 0.008*"great" + 0.007*"food" + 0.007*"good" + 0.007*"service" + 0.007*"come" + 0.007*"like" + 0.006*"order"'),
 (2,
  '0.042*"" + 0.008*"place" + 0.007*"time" + 0.007*"food" + 0.007*"order" + 0.007*"come" + 0.007*"great" + 0.006*"like" + 0.006*"service" + 0.006*"want"'),
 (3,
  '0.050*"" + 0.009*"place" + 0.009*"food" + 0.007*"good" + 0.007*"get" + 0.007*"time" + 0.006*"come" + 0.005*"like" + 0.005*"go" + 0.005*"great"'),
 (4,
  '0.055*"" + 0.009*"like" + 0.009*"food" + 0.007*"place" + 0.007*"good" + 0.006*"time" + 0.006*"come" + 0.006*"well" + 0.006*"order" + 0.005*"great"'),
 (5,
  '0.051*"" + 0.011*"place" + 0.010*"good" + 0.009*"time" + 0.008*"like" + 0.008*"great" + 0.007*"service" + 0.007*"food" + 0.006*"go" + 0.005*"come"'),
 (6,
  '0.052*"" + 0.009*"time" + 0.009*"good" + 

Create 1-2 visualizations of the results

In [75]:

words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:7]) for t in words]

In [76]:
print(topics)

[' place food good come time order', ' place time great food good service', ' place time food order come great', ' place food good get time come', ' like food place good time come', ' place good time like great service', ' time good place food like order']


In [77]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
 place food good come time order

------ Topic 1 ------
 place time great food good service

------ Topic 2 ------
 place time food order come great

------ Topic 3 ------
 place food good get time come

------ Topic 4 ------
 like food place good time come

------ Topic 5 ------
 place good time like great service

------ Topic 6 ------
 time good place food like order



In [79]:
pyLDAvis.enable_notebook()

In [80]:
%%time
pyLDAvis.gensim.prepare(lda, corpus, dictionary=id2word)

Wall time: 7min 21s


d:\csfiles\lambda\git\nlp\venv\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.003668  0.003291       1        1  15.041788
1      0.000559  0.001410       2        1  14.523438
6     -0.000017 -0.003735       3        1  14.390011
4      0.000299 -0.002858       4        1  14.156954
5     -0.001889 -0.000423       5        1  14.042290
3     -0.000772  0.000228       6        1  13.987473
2      0.005489  0.002086       7        1  13.858053, topic_info=    Category          Freq        Term         Total  loglift  logprob
307  Default   1727.000000        want   1727.000000  30.0000  30.0000
24   Default   3559.000000        like   3559.000000  29.0000  29.0000
50   Default   4171.000000        good   4171.000000  28.0000  28.0000
130  Default   5005.000000       place   5005.000000  27.0000  27.0000
0    Default  26398.000000              26398.000000  26.0000  26.0000
..       ...           ...         ...           ...      ...      ...
50    Topic7    411.934875        good   4171.502930  -0.3389  -5.1661
473   Topic7    216.725433        know   1618.331177  -0.0342  -5.8084
448   Topic7    205.360947         day   1447.437500   0.0235  -5.8622
98    Topic7    207.626770        nice   1594.738281  -0.0624  -5.8513
106   Topic7    204.117264  restaurant   1603.162354  -0.0847  -5.8683

[630 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
0          1  0.152696        
0          2  0.153226        
0          3  0.148150        
0          4  0.152544        
0          5  0.140612        
...      ...       ...     ...
14202      2  0.156500  zombie
14202      4  0.156500  zombie
14202      5  0.312999  zombie
14202      6  0.156500  zombie
14202      7  0.156500  zombie

[1986 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 7, 5, 6, 4, 3])

**Conclusion**:

It looks like for the majority of topics in the bag of words, things were either talking about places, food, time, or service. This would make sense because this is a Yelp dataset, after all. The majority of topics are going to be about that.

There seems to be a positive association in general, but since this is a BoW model, we can't be too sure. This would indicate that most reviews are positive, or at the very least, most reviews mention some positive aspect of the place they are reviewing.

However, there is so much overlap between the topics, that further data cleaning and removal would probably be the best route to go with before further analysis.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)